This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-ml-operations/packaging-and-deployment/packaging-and-deployment-by-framework/xgboost).

## XGBoost RF Classification Deployment Tutorial

The following tutorial demonstrates how to upload a XGBoost RF Classification model to a Wallaroo instance.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Custom Models, also known as Bring Your Own Predict (BYOP)](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

### Tutorial Goals

Demonstrate the following:

* Upload a XGBoost RF Classification model to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* A Wallaroo version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [1]:
from sklearn.datasets import load_iris
from xgboost import XGBClassifier

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'xgboost-rf-classification'
pipeline_name = f'xgboost-rf-classification'

model_name = 'xgboost-rf-classification'
model_file_name = './models/xgb_rf_classifier.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

XGBoost models are uploaded to Wallaroo through the Wallaroo Client `upload_model` method.

### Upload XGBoost Model Parameters

The following parameters are available for XGBoost models.  

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Required*) | Set as `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### XGBoost Schema Inputs

XGBoost schema follows a different format than other models.  To prevent inputs from being out of order, the inputs should be submitted in a single row in the order the model is trained to accept, with **all of the data types being the same**.  If a model is originally trained to accept inputs of different data types, it will need to be retrained to only accept one data type for each column - typically `pa.float64()` is a good choice.

For example, the following DataFrame has 4 columns, each column a `float`.

|&nbsp;|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|
|---|---|---|---|---|
|0|5.1|3.5|1.4|0.2|
|1|4.9|3.0|1.4|0.2|

For submission to an XGBoost model, the data input schema will be a single array with 4 float values.

```python
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float64(), list_size=4))
])
```

When submitting as an inference, the DataFrame is converted to rows with the column data expressed as a single array.  The data **must** be in the same order as the model expects, which is why the data is submitted as a single array rather than JSON labeled columns:  this insures that the data is submitted in the exact order as the model is trained to accept.

Original DataFrame:

&nbsp;|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)
|---|---|---|---|---|
0|5.1|3.5|1.4|0.2
1|4.9|3.0|1.4|0.2

Converted DataFrame:

|&nbsp;|inputs|
|---|---|
|0|[5.1, 3.5, 1.4, 0.2]|
|1|[4.9, 3.0, 1.4, 0.2]|

#### XGBoost Schema Outputs

Outputs for XGBoost are labeled based on the trained model outputs.  For this example, the output is simply a single output listed as `output`.  In the Wallaroo inference result, it is grouped with the metadata `out` as `out.output`.

```python
output_schema = pa.schema([
    pa.field('output', pa.int32())
])
```

```python
pipeline.infer(dataframe)
```

|&nbsp;|time|in.inputs|out.output|check_failures|
|---|---|---|---|---|
|0|2023-07-05 15:11:29.776|[5.1, 3.5, 1.4, 0.2]|0|0|
|1|2023-07-05 15:11:29.776|[4.9, 3.0, 1.4, 0.2]|0|0|


In [5]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=4))
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
    pa.field('probabilities', pa.list_(pa.float32(), list_size=3))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.XGBOOST`.

In [7]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime...incompatible

Model is pending loading to a container runtime..
Model is attempting loading to a container runtime...successful

Ready


Name,xgboost-rf-classification
Version,0fcd0d49-fdb7-4ee0-a599-ba0f1394ab04
File Name,xgb_rf_classifier.pkl
SHA,1e5ecfd77cb3bb75eab7aaf5b3975d037191c2d9e825805bff027fbec76be296
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.2.0-main-5408
Architecture,x86
Acceleration,none
Updated At,2024-19-Jul 18:00:13
Workspace id,45
Workspace name,xgboost-rf-classification


### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [9]:
# clear the pipeline if it was used before
pipeline.undeploy()
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)

name,xgboost-rf-classification
created,2024-07-19 16:39:08.601984+00:00
last_updated,2024-07-19 18:00:21.961555+00:00
deployed,True
workspace_id,45
workspace_name,xgboost-rf-classification
arch,x86
accel,none
tags,
versions,"7bac25f2-74e1-4166-97a1-e92f382741d5, b61bb623-dfde-4676-b48b-4178096f172c, 7d19728f-6a44-4858-b11e-6c9029622f83, 335aebc6-5a6a-4563-a039-bbbd54c7c060, 7bae5762-5e2b-4d95-8c95-b01003438330, 1d45cd70-06a5-4570-84b6-93d7ce3f8139, 6f555bf1-36c0-4b49-84ff-9b72baf3020b, 14653559-ce12-4ebb-912d-22dbc21fec7c, 94df97ae-5594-4b65-829f-ad8eb4df5c9d, ff951a29-2a5c-4855-aa74-57350a9dda11, 77010639-8d71-48be-9403-3ef0e67cfb0f"
steps,xgboost-rf-classification


### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [10]:
data = load_iris(as_frame=True)

X = data['data'].values[:100]
y = data['target'][:100]

dataframe = pd.DataFrame({"inputs": X[:100].tolist()})
dataframe

,inputs
0,"[5.1, 3.5, 1.4, 0.2]"
1,"[4.9, 3.0, 1.4, 0.2]"
2,"[4.7, 3.2, 1.3, 0.2]"
3,"[4.6, 3.1, 1.5, 0.2]"
4,"[5.0, 3.6, 1.4, 0.2]"
...,...
95,"[5.7, 3.0, 4.2, 1.2]"
96,"[5.7, 2.9, 4.2, 1.3]"
97,"[6.2, 2.9, 4.3, 1.3]"
98,"[5.1, 2.5, 3.0, 1.1]"


In [11]:
pipeline.infer(dataframe)

,time,in.inputs,out.predictions,out.probabilities,anomaly.count
0,2024-07-19 18:00:45.231,"[5.1, 3.5, 1.4, 0.2]",0,"[0.82590103, 0.087049514, 0.08704949]",0
1,2024-07-19 18:00:45.231,"[4.9, 3.0, 1.4, 0.2]",0,"[0.82590103, 0.087049514, 0.08704949]",0
2,2024-07-19 18:00:45.231,"[4.7, 3.2, 1.3, 0.2]",0,"[0.82590103, 0.087049514, 0.08704949]",0
3,2024-07-19 18:00:45.231,"[4.6, 3.1, 1.5, 0.2]",0,"[0.82590103, 0.087049514, 0.08704949]",0
4,2024-07-19 18:00:45.231,"[5.0, 3.6, 1.4, 0.2]",0,"[0.82590103, 0.087049514, 0.08704949]",0
...,...,...,...,...,...
95,2024-07-19 18:00:45.231,"[5.7, 3.0, 4.2, 1.2]",1,"[0.08704949, 0.82590103, 0.08704949]",0
96,2024-07-19 18:00:45.231,"[5.7, 2.9, 4.2, 1.3]",1,"[0.08704949, 0.82590103, 0.08704949]",0
97,2024-07-19 18:00:45.231,"[6.2, 2.9, 4.3, 1.3]",1,"[0.08704949, 0.82590103, 0.08704949]",0
98,2024-07-19 18:00:45.231,"[5.1, 2.5, 3.0, 1.1]",1,"[0.088133186, 0.82373357, 0.088133186]",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [12]:
pipeline.undeploy()

name,xgboost-rf-classification
created,2024-07-19 16:39:08.601984+00:00
last_updated,2024-07-19 18:00:21.961555+00:00
deployed,False
workspace_id,45
workspace_name,xgboost-rf-classification
arch,x86
accel,none
tags,
versions,"7bac25f2-74e1-4166-97a1-e92f382741d5, b61bb623-dfde-4676-b48b-4178096f172c, 7d19728f-6a44-4858-b11e-6c9029622f83, 335aebc6-5a6a-4563-a039-bbbd54c7c060, 7bae5762-5e2b-4d95-8c95-b01003438330, 1d45cd70-06a5-4570-84b6-93d7ce3f8139, 6f555bf1-36c0-4b49-84ff-9b72baf3020b, 14653559-ce12-4ebb-912d-22dbc21fec7c, 94df97ae-5594-4b65-829f-ad8eb4df5c9d, ff951a29-2a5c-4855-aa74-57350a9dda11, 77010639-8d71-48be-9403-3ef0e67cfb0f"
steps,xgboost-rf-classification
